# Analysis-of-2024-French-Legislatives-Elections

İmport Modules

In [12]:
import pandas as pd
import os 
import sqlite3
import google.generativeai as genai
import csv
import dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain.agents import initialize_agent , tools 



# Gathering Data

In [13]:
csv_file = 'resultats-definitifs-par-regions.csv'
df = pd.read_csv(csv_file, sep=';', decimal=',')

# create a SQLite database and connect to it
db_file = 'elections.db'
conn = sqlite3.connect(db_file)

# write the DataFrame to a SQLite table
df.to_sql('elections', conn, if_exists='replace', index=False)
conn.close()
print(f"Data from {csv_file} has been written to {db_file} in the 'elections' table.")


Data from resultats-definitifs-par-regions.csv has been written to elections.db in the 'elections' table.


# Using the LLM to Write and Run SQL

In [14]:
import os
import dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_experimental.sql import SQLDatabaseChain
from langchain_community.utilities import SQLDatabase
from langchain.agents import initialize_agent, Tool # Import Tool


dotenv.load_dotenv(override=True)

# Database setup
db = SQLDatabase.from_uri("sqlite:///elections.db")


llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-preview-04-17",
    temperature=0,
    google_api_key=os.getenv("GOOGLE_API_KEY")
)


db_chain = SQLDatabaseChain(
    llm=llm,
    database=db,
    verbose=True,
    return_direct=True 
)
tools = [
    Tool(
        name="Elections Database",
        func=db_chain.run, 
        description="Useful for answering questions about elections data. Input should be a question."
    )
]




agent = initialize_agent(
    tools, 
    llm,
    agent="zero-shot-react-description",
    max_iterations=1,
    early_stopping_method="generate",
    verbose=True
)

try:
    response = agent.run("How many voters are in France?")
    print(response)
except Exception as e:
    print(f"Agent execution failed: {e}")




> Entering new AgentExecutor chain...


c:\Users\batud\AppData\Local\Programs\Python\Python313\Lib\site-packages\langchain_experimental\sql\base.py:77: UserWarning: Directly instantiating an SQLDatabaseChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


The user is asking a question about the number of voters in France. The available tool is an "Elections Database". This tool seems relevant for answering questions about elections data, which likely includes information about voters. I should use this tool to find the number of voters in France.Action: Elections Database
Action Input: How many voters are in France?

> Entering new SQLDatabaseChain chain...
How many voters are in France?
SQLQuery:SQLQuery: SELECT SUM("Votants") FROM elections
SQLResult: [(32093013,)]
> Finished chain.

Observation: [(32093013,)]
Thought:I now know the final answer
Final Answer: There are 32,093,013 voters in France.

> Finished chain.
There are 32,093,013 voters in France.


In [15]:
db_chain("Est-ce que tu peux trouver le nombre de votants au Rassemblement National?")

C:\Users\batud\AppData\Local\Temp\ipykernel_22080\671110524.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  db_chain("Est-ce que tu peux trouver le nombre de votants au Rassemblement National?")




> Entering new SQLDatabaseChain chain...
Est-ce que tu peux trouver le nombre de votants au Rassemblement National?
SQLQuery:Question: Est-ce que tu peux trouver le nombre de votants au Rassemblement National?
SQLQuery: SELECT SUM(CASE WHEN "Nuance candidat 1" = 'RN' THEN "Voix 1" ELSE 0 END) + SUM(CASE WHEN "Nuance candidat 2" = 'RN' THEN "Voix 2" ELSE 0 END) + SUM(CASE WHEN "Nuance candidat 3" = 'RN' THEN "Voix 3" ELSE 0 END) + SUM(CASE WHEN "Nuance candidat 4" = 'RN' THEN "Voix 4" ELSE 0 END) + SUM(CASE WHEN "Nuance candidat 5" = 'RN' THEN "Voix 5" ELSE 0 END) + SUM(CASE WHEN "Nuance candidat 6" = 'RN' THEN "Voix 6" ELSE 0 END) + SUM(CASE WHEN "Nuance candidat 7" = 'RN' THEN "Voix 7" ELSE 0 END) + SUM(CASE WHEN "Nuance candidat 8" = 'RN' THEN "Voix 8" ELSE 0 END) + SUM(CASE WHEN "Nuance candidat 9" = 'RN' THEN "Voix 9" ELSE 0 END) + SUM(CASE WHEN "Nuance candidat 10" = 'RN' THEN "Voix 10" ELSE 0 END) + SUM(CASE WHEN "Nuance candidat 11" = 'RN' THEN "Voix 11" ELSE 0 END) + SUM(CASE

{'query': 'Est-ce que tu peux trouver le nombre de votants au Rassemblement National?',
 'result': '[(9316716.0,)]'}

In [17]:
db_chain("parisin olduğu bölgede milliyetçi hareket partisi kaç oy olmış toplam")



> Entering new SQLDatabaseChain chain...
parisin olduğu bölgede milliyetçi hareket partisi kaç oy olmış toplam
SQLQuery:Question: parisin olduğu bölgede milliyetçi hareket partisi kaç oy olmış toplam
SQLQuery: SELECT SUM(CASE WHEN "Nuance candidat 1" = 'RN' THEN "Voix 1" ELSE 0 END + CASE WHEN "Nuance candidat 2" = 'RN' THEN "Voix 2" ELSE 0 END + CASE WHEN "Nuance candidat 3" = 'RN' THEN "Voix 3" ELSE 0 END + CASE WHEN "Nuance candidat 4" = 'RN' THEN "Voix 4" ELSE 0 END + CASE WHEN "Nuance candidat 5" = 'RN' THEN "Voix 5" ELSE 0 END + CASE WHEN "Nuance candidat 6" = 'RN' THEN "Voix 6" ELSE 0 END + CASE WHEN "Nuance candidat 7" = 'RN' THEN "Voix 7" ELSE 0 END + CASE WHEN "Nuance candidat 8" = 'RN' THEN "Voix 8" ELSE 0 END + CASE WHEN "Nuance candidat 9" = 'RN' THEN "Voix 9" ELSE 0 END + CASE WHEN "Nuance candidat 10" = 'RN' THEN "Voix 10" ELSE 0 END + CASE WHEN "Nuance candidat 11" = 'RN' THEN "Voix 11" ELSE 0 END + CASE WHEN "Nuance candidat 12" = 'RN' THEN "Voix 12" ELSE 0 END + CAS

{'query': 'parisin olduğu bölgede milliyetçi hareket partisi kaç oy olmış toplam',
 'result': '[(818552.0,)]'}